In [ ]:
!pip install pymssql

In [ ]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde as kde
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

In [ ]:

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [ ]:

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data['rnk'] = data.ret.rank(pct=True)

In [ ]:

transform = QuantileTransformer(
    output_distribution="normal"
)

old = data.roeq.to_numpy().reshape(-1,1)
new = transform.fit_transform(old)

old = np.ravel(old)
new = np.ravel(new)

trace = go.Box(
    y=old,
    name="old",
)

fig = go.Figure(trace)

fig.update_layout(
    template="plotly_dark",
    yaxis_tickformat=".0%",
)
fig.show()

In [ ]:

trace = go.Box(
    y=new,
    name="new",
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    yaxis_tickformat=".0%",
)
fig.show()

In [ ]:

density = kde(old)
mn = np.min(old)
mx = np.max(old)
grid = np.linspace(mn, mx, 100)

trace = go.Scatter(
  x=grid, 
  y=density(grid),
  mode="lines", 
  name="old"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    yaxis_title="",
    xaxis_tickformat=".0%",
    yaxis_tickformat="",
)

fig.show()

In [ ]:
density = kde(new)
mn = np.min(new)
mx = np.max(new)
grid = np.linspace(mn, mx, 100)

trace = go.Scatter(
  x=grid, 
  y=density(grid),
  mode="lines", 
  name="new"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    yaxis_title="",
    xaxis_tickformat=".0%",
    yaxis_tickformat="",
)

fig.show()

In [ ]:
model = MLPRegressor(
  hidden_layer_sizes=(4, 2),
  random_state=0
)

poly = PolynomialFeatures(degree=2)
pipe = make_pipeline(
  transform, 
  poly,
  transform,
  model
)
X = data[["roeq", "mom12m"]]
y = data["rnk"]

pipe.fit(X, y)
dump(pipe, "net2.joblib")